In [0]:
import pandas as pd

# Importing Rhea

In [0]:
rhea2xref = pd.read_csv("ftp://ftp.expasy.org/databases/rhea/tsv/rhea2xrefs.tsv", sep="\t")

In [3]:
rhea2xref.head()

,RHEA_ID,DIRECTION,MASTER_ID,ID,DB
0,10000,UN,10000,3.5.1.50,EC
1,10003,BI,10000,R02938,KEGG_REACTION
2,10003,BI,10000,PENTANAMIDASE-RXN,METACYC
3,10004,UN,10004,5.99.1.1,EC
4,10007,BI,10004,R04010,KEGG_REACTION


In [0]:
kegg2xref = rhea2xref[rhea2xref["DB"] == "KEGG_REACTION"]

In [5]:
kegg2xref.head()

,RHEA_ID,DIRECTION,MASTER_ID,ID,DB
1,10003,BI,10000,R02938,KEGG_REACTION
4,10007,BI,10004,R04010,KEGG_REACTION
7,10011,BI,10008,R07180,KEGG_REACTION
11,10015,BI,10012,R07170,KEGG_REACTION
14,10019,BI,10016,R02381,KEGG_REACTION


In [0]:
ecocyc2xref = rhea2xref[rhea2xref["DB"] == "ECOCYC"]

In [7]:
ecocyc2xref.head()

,RHEA_ID,DIRECTION,MASTER_ID,ID,DB
32,10041,LR,10040,325-BISPHOSPHATE-NUCLEOTIDASE-RXN,ECOCYC
73,10093,LR,10092,DEOXYNUCLEOTIDE-3-PHOSPHATASE-RXN,ECOCYC
93,10117,LR,10116,RXN-9952,ECOCYC
99,10121,LR,10120,AMINOCYL-TRNA-HYDROLASE-RXN,ECOCYC
113,10137,LR,10136,GLYOCARBOLIG-RXN,ECOCYC


# How many lipid reactions in kegg2xref?

In [0]:
keggtext = "00061Fatty acid biosynthesis00062Fatty acid elongation00071Fatty acid degradation00072Synthesis and degradation of ketone bodies00073Cutin, suberine and wax biosynthesis00100Steroid biosynthesis00120Primary bile acid biosynthesis00121Secondary bile acid biosynthesis00140Steroid hormone biosynthesis00561Glycerolipid metabolism00564Glycerophospholipid metabolism00565Ether lipid metabolism00600Sphingolipid metabolism00590Arachidonic acid metabolism00591Linoleic acid metabolism00592alpha-Linolenic acid metabolism01040Biosynthesis of unsaturated fatty acids"

In [0]:
import re
pathwayIds = re.findall(r"[0-9]{5}", keggtext)


In [10]:
pathwayIds

['00061',
 '00062',
 '00071',
 '00072',
 '00073',
 '00100',
 '00120',
 '00121',
 '00140',
 '00561',
 '00564',
 '00565',
 '00600',
 '00590',
 '00591',
 '00592',
 '01040']

In [0]:
lipidmaps = pd.DataFrame()

In [12]:
for id in pathwayIds:
  try:
    lipidmaps = pd.concat([lipidmaps, pd.read_csv("http://rest.kegg.jp/link/rn/map" + id, sep="\t", header=None)])
  except pd.errors.EmptyDataError:
    print("No reaction ID in map" + id)

No reaction ID in map00121


In [13]:
lipidmaps.head()

,0,1
0,path:map00061,rn:R00742
1,path:map00061,rn:R01280
2,path:map00061,rn:R01624
3,path:map00061,rn:R01626
4,path:map00061,rn:R01706


In [0]:
lipidRids = lipidmaps[1].str.replace("rn:", "")

In [0]:
lipidkegg2xref = kegg2xref[kegg2xref["ID"].isin(lipidRids)]

In [16]:
print(kegg2xref["ID"].unique().shape)
print(lipidkegg2xref["ID"].unique().shape)

(6157,)
(351,)


# KEGG lipid reactions in Rhea have EcoCyc reaction IDs?

In [0]:
lipidkegg2xref = rhea2xref[rhea2xref["RHEA_ID"].isin(lipidkegg2xref["RHEA_ID"])]

In [36]:
lipidkegg2xref.head()

,RHEA_ID,DIRECTION,MASTER_ID,ID,DB
161,10191,BI,10188,R01978,KEGG_REACTION
162,10191,BI,10188,HYDROXYMETHYLGLUTARYL-COA-SYNTHASE-RXN,METACYC
231,10275,BI,10272,R02240,KEGG_REACTION
232,10275,BI,10272,DIACYLGLYKIN-RXN,ECOCYC
233,10275,BI,10272,DIACYLGLYKIN-RXN,METACYC


In [41]:
kegg = lipidkegg2xref["DB"] == "KEGG_REACTION"
kegg.value_counts()

True     362
False    108
Name: DB, dtype: int64

In [38]:
metacyc = lipidkegg2xref["DB"] == "METACYC"
metacyc.value_counts()

False    373
True      97
Name: DB, dtype: int64

In [39]:
ecocyc = lipidkegg2xref["DB"] == "ECOCYC"
ecocyc.value_counts()

False    459
True      11
Name: DB, dtype: int64